TODO: Split the annotating of mutations with conditions and the associating of genomic features with conditions into 2 NBs.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from IPython.display import display
import os, sys, itertools, csv
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from util.alemutdf import get_all_sample_mut_df, get_gene_mut_count_mat, get_multi_exp_max_freq_mut_df, get_mut_type_avg_frac_across_class_df
from util.metadata import get_condition_val_dict, get_condition_field_val_set
from util.genome import get_K12_pos_from_BOP27
from util.params import ASSOC_ALPHA, MULTI_HYP_CORR_METHOD
pd.options.display.max_columns = 100

In [2]:
# starting efforts for refactoring of assoc NBs.
FEAT_ANNOT = "genomic features"

In [3]:
all_muts_df = pd.read_pickle("./data/2_9_df.pkl")
display(all_muts_df.shape, all_muts_df.head())

(5676, 42)

,index,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,mutation target annotation,Reference Seq,sample,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links,pathways,pathway links,COGs,COG links,imodulons,imodulon links,EC numbers,EC number links,products,product links
0,9,42C,1,124,1,1,1.0,702352,DEL,Δ21 bp,coding (380‑400/1149 nt),nagA,NaN,1 124 1 1,True,"(702352, 702372)",{ECK120000625},"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",False,False,{},{},{},{},{},True,"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",{'ECK120000625': ['ECK120000625']},"[{'name': 'nagBAC-umpH', 'RegulonDB ID': 'ECK1...",{'ECK125162049': ['ECK120000625']},"[{'name': 'PhoP'}, {'name': 'NagC'}, {'name': ...","{'NagC': ['ECK125162049'], 'PhoP': ['ECK125162...","[{'name': 'Galactose metabolism'}, {'name': 'A...","{'Galactose metabolism': ['ECK125162049'], 'Am...",[{'name': 'Carbohydrate transport and metaboli...,{'Carbohydrate transport and metabolism': ['EC...,"[{'name': 'NagC/TyrR'}, {'name': 'NagC/TyrR'}]",{'NagC/TyrR': ['nagBAC-umpH']},"[{'name': '3.5.1.25'}, {'name': '3.5.1.25'}]","{'3.5.1.25': ['ECK125162049', 'ECK125162049']}",[{'name': 'N-acetylglucosamine-6-phosphate dea...,{'N-acetylglucosamine-6-phosphate deacetylase ...
1,10,42C,1,124,1,1,1.0,1308318,SNP,G→C,R110G (CGT→GGT),clsA,NaN,1 124 1 1,True,"(1308318, 1308318)",{ECK120001556},"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",False,False,{},{},{},{},{},True,"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",{'ECK120001556': ['ECK120001556']},"[{'name': 'clsA-yciU', 'RegulonDB ID': 'ECK125...",{'ECK125147415': ['ECK120001556']},[],{},[],{},[{'name': 'Lipid transport and metabolism'}],{'Lipid transport and metabolism': ['ECK125147...,[],{},[],{},[],{}
2,11,42C,1,124,1,1,1.0,3815859,DEL,Δ82 bp,,rph,NaN,1 124 1 1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK120000854'], 'ECK1200007...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000079...",[],{},"[{'name': 'Pyrimidine metabolism'}, {'name': '...","{'Pyrimidine metabolism': ['ECK120014627'], 'D...",[{'name': 'Nucleotide transport and metabolism'}],{'Nucleotide transport and metabolism': ['ECK1...,"[{'name': 'purR-KO'}, {'name': 'purR-KO'}]","{'purR-KO': ['rph-pyrE', 'rph-pyrE']}","[{'name': '2.4.2.10'}, {'name': '2.4.2.10'}]","{'2.4.2.10': ['ECK120014627', 'ECK120014627']}",[{'name': 'Orotate phosphoribosyltransferase (...,{'Orotate phosphoribosyltransferase (EC 2.4.2....
3,12,42C,1,124,1,1,1.0,4187550,SNP,C→T,A734V (GCG→GTG),rpoC,NaN,1 124 1 1,True,"(4187550, 4187550)",{ECK120000886},"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",False,False,{},{},{},{},{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[],{},"[{'name': 'Purine metabolism'}, {'name': 'Pyri...","{'Purine metabolism': ['ECK120016992'], 'Pyrim...",[{'name': 'Transcription'}],{'Transcription': ['ECK120016992']},[{'name': 'translation'}],{'translation': ['rplKAJL-rpoBC']},"[{'name': '2.7.7.6'}, {'name': '2.7.7.6'}]","{'2.7.7.6': ['ECK120016992', 'ECK120016992']}",[{'name': 'DNA-directed RNA polymerase beta' s...,{'DNA-directed RNA polymerase beta' subunit (E...
4,13,42C,1,124,1,1,1.0,4400313,SNP,A→C,D9A (GAT→GCT),hfq,NaN,1 124 1 1,True,"(4400313, 4400313)",{ECK120000431},"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",False,False,{},{},{},{},{},True,"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",{'ECK120000431': ['ECK120000431']},[{'name': 'yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC...,{'ECK125147410': ['ECK120000431']},[{'nam

In [4]:
# temporarily changing some of the contents of the mutation data since 
temp_exp_to_remove_till_reprocess = ["42C_Tenaillon", "TEE", "TEE_Ara", "LTEE", "LTEE_Ara", "delta_dnaQ"]
all_muts_df = all_muts_df[~(all_muts_df.exp.isin(temp_exp_to_remove_till_reprocess))]
all_muts_df["exp"] = all_muts_df.apply(lambda r: "gallium-nitrate" if r["exp"] == "untreated" else r["exp"], axis=1)

## enrichment analysis between conditions and mutated genomic features

### Get metadata

In [1]:
from util.metadata import get_all_exp_cond_d

all_exp_cond_d = get_all_exp_cond_d("./data/metadata/")
all_exp_cond_d["TOL_2,3-butanediol"] = all_exp_cond_d.pop("TOL_2 3-butanediol")  # TODO: workaround for bug of missing comma. Fix root bug
all_exp_cond_d

{'Iron-ALE': {'taxonomy-id': '511145',
  'strain-description': 'WT',
  'temperature': '37 celsius',
  'base-media': 'M9',
  'carbon-source': 'glucose(2)',
  'nitrogen-source': 'NH4Cl(1)',
  'phosphorous-source': 'KH2PO4(3) Na2HPO4(6.8)',
  'sulfur-source': 'MgSO4(0.24)',
  'supplement': 'FeSO4(20uM) sodium citrate(10mM)',
  'calcium-source': 'CaCl2(0.1)'},
 'application_of_ALE_to_re-optimize_growth_performance_of_genome-reduced_strain_MS56-M9-MG1655gbk': {'carbon-source': 'glucose(2)',
  'taxonomy-id': '511145',
  'strain-description': 'MS57',
  'temperature': '37 celsius',
  'base-media': 'M63',
  'supplement': 'NaCl(0.5g/L) trace elements',
  'nitrogen-source': 'NH4Cl(1)',
  'phosphorous-source': 'KH2PO4(3) Na2HPO4(6.78)',
  'sulfur-source': 'MgSO4(1mM)',
  'calcium-source': 'CaCl2(0.1mM)'},
 'pvdJ_KO_P_aeruginosa_spent_medium': {'carbon-source': 'glucose(3)',
  'taxonomy-id': '511145',
  'strain-description': 'WT',
  'temperature': '37 celsius',
  'base-media': 'M63',
  'supplement'

In [6]:
# So that rows will always contain the same amount of columns
all_muts_df = all_muts_df.replace('', "None")
display(all_muts_df.shape, all_muts_df.head())

(5676, 42)

,index,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,mutation target annotation,Reference Seq,sample,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links,pathways,pathway links,COGs,COG links,imodulons,imodulon links,EC numbers,EC number links,products,product links
0,9,42C,1,124,1,1,1.0,702352,DEL,Δ21 bp,coding (380‑400/1149 nt),nagA,NaN,1 124 1 1,True,"(702352, 702372)",{ECK120000625},"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",False,False,{},{},{},{},{},True,"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",{'ECK120000625': ['ECK120000625']},"[{'name': 'nagBAC-umpH', 'RegulonDB ID': 'ECK1...",{'ECK125162049': ['ECK120000625']},"[{'name': 'PhoP'}, {'name': 'NagC'}, {'name': ...","{'NagC': ['ECK125162049'], 'PhoP': ['ECK125162...","[{'name': 'Galactose metabolism'}, {'name': 'A...","{'Galactose metabolism': ['ECK125162049'], 'Am...",[{'name': 'Carbohydrate transport and metaboli...,{'Carbohydrate transport and metabolism': ['EC...,"[{'name': 'NagC/TyrR'}, {'name': 'NagC/TyrR'}]",{'NagC/TyrR': ['nagBAC-umpH']},"[{'name': '3.5.1.25'}, {'name': '3.5.1.25'}]","{'3.5.1.25': ['ECK125162049', 'ECK125162049']}",[{'name': 'N-acetylglucosamine-6-phosphate dea...,{'N-acetylglucosamine-6-phosphate deacetylase ...
1,10,42C,1,124,1,1,1.0,1308318,SNP,G→C,R110G (CGT→GGT),clsA,NaN,1 124 1 1,True,"(1308318, 1308318)",{ECK120001556},"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",False,False,{},{},{},{},{},True,"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",{'ECK120001556': ['ECK120001556']},"[{'name': 'clsA-yciU', 'RegulonDB ID': 'ECK125...",{'ECK125147415': ['ECK120001556']},[],{},[],{},[{'name': 'Lipid transport and metabolism'}],{'Lipid transport and metabolism': ['ECK125147...,[],{},[],{},[],{}
2,11,42C,1,124,1,1,1.0,3815859,DEL,Δ82 bp,None,rph,NaN,1 124 1 1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK120000854'], 'ECK1200007...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000079...",[],{},"[{'name': 'Pyrimidine metabolism'}, {'name': '...","{'Pyrimidine metabolism': ['ECK120014627'], 'D...",[{'name': 'Nucleotide transport and metabolism'}],{'Nucleotide transport and metabolism': ['ECK1...,"[{'name': 'purR-KO'}, {'name': 'purR-KO'}]","{'purR-KO': ['rph-pyrE', 'rph-pyrE']}","[{'name': '2.4.2.10'}, {'name': '2.4.2.10'}]","{'2.4.2.10': ['ECK120014627', 'ECK120014627']}",[{'name': 'Orotate phosphoribosyltransferase (...,{'Orotate phosphoribosyltransferase (EC 2.4.2....
3,12,42C,1,124,1,1,1.0,4187550,SNP,C→T,A734V (GCG→GTG),rpoC,NaN,1 124 1 1,True,"(4187550, 4187550)",{ECK120000886},"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",False,False,{},{},{},{},{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[],{},"[{'name': 'Purine metabolism'}, {'name': 'Pyri...","{'Purine metabolism': ['ECK120016992'], 'Pyrim...",[{'name': 'Transcription'}],{'Transcription': ['ECK120016992']},[{'name': 'translation'}],{'translation': ['rplKAJL-rpoBC']},"[{'name': '2.7.7.6'}, {'name': '2.7.7.6'}]","{'2.7.7.6': ['ECK120016992', 'ECK120016992']}",[{'name': 'DNA-directed RNA polymerase beta' s...,{'DNA-directed RNA polymerase beta' subunit (E...
4,13,42C,1,124,1,1,1.0,4400313,SNP,A→C,D9A (GAT→GCT),hfq,NaN,1 124 1 1,True,"(4400313, 4400313)",{ECK120000431},"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",False,False,{},{},{},{},{},True,"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",{'ECK120000431': ['ECK120000431']},[{'name': 'yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC...,{'ECK125147410': ['ECK120000431']},[{

In [7]:
all_muts_df.exp.unique()

array(['42C', 'C13', 'EG_pCA24N-fucO', 'GLU', 'GYD', 'ILT_MG1655',
       'Iron-ALE', 'JCL260__isobutanol_production_host-M9-MG1655gbk',
       'MG1655-M9-NC_000913_3gb-stationary-37-D-2-deoxyribose2',
       'MG1655-M9-NC_000913_3gb-stationary-37-D-arabinose2',
       'MG1655-M9-NC_000913_3gb-stationary-37-D-lyxose2',
       'MG1655-M9-NC_000913_3gb-stationary-37-m-tartrate2',
       'MG1655-M9-NC_000913_3gb-stationary-37-monomethyl_succinate2',
       'MG1655-blaTEM1_ceftazidime', 'MG1655-blaTEM1_untreated',
       'MG1655-pBGT_ceftazidime', 'MG1655-pBGT_untreated',
       'MG1655_ceftazidime', 'MG1655_chloramphenicol',
       'MG1655_doxycycline', 'MG1655_trimethoprim', 'MG1655_untreated',
       'OxidizeME_PQ3', 'PGI', 'SER', 'SSW_AC', 'SSW_GLU_AC',
       'SSW_GLU_GLY', 'SSW_GLU_XYL', 'SSW_GLY', 'SSW_XYL',
       'TOL_2,3-butanediol', 'TOL_adipic_acid', 'TOL_coumaric_acid',
       'TOL_glutaric_acid', 'TOL_hexamethylenediamine',
       'TOL_hexanoic_acid', 'TOL_isobutyric_acid', '

### mutation to condition association

In [8]:
for exp, md_d in all_exp_cond_d.items():
    if "calcium-source" not in md_d.keys():
        print(exp)

In [9]:
from util.metadata_categories_for_associations import METADATA_CATEGORIES_FOR_ASSOCIATIONS
print(len(METADATA_CATEGORIES_FOR_ASSOCIATIONS))

for condition in METADATA_CATEGORIES_FOR_ASSOCIATIONS:
    all_muts_df[condition] = all_muts_df["exp"].apply(lambda exp: all_exp_cond_d[exp][condition])
    
all_muts_df = all_muts_df.reset_index(drop=True)

10


In [10]:
# The below is the necessary logic without this filtering.
exp_target_cond_df = all_muts_df.copy()
display(exp_target_cond_df.shape, exp_target_cond_df.head())

(5676, 52)

,index,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,Details,mutation target annotation,Reference Seq,sample,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links,pathways,pathway links,COGs,COG links,imodulons,imodulon links,EC numbers,EC number links,products,product links,temperature,carbon-source,supplement,strain-description,taxonomy-id,base-media,nitrogen-source,phosphorous-source,sulfur-source,calcium-source
0,9,42C,1,124,1,1,1.0,702352,DEL,Δ21 bp,coding (380‑400/1149 nt),nagA,NaN,1 124 1 1,True,"(702352, 702372)",{ECK120000625},"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",False,False,{},{},{},{},{},True,"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",{'ECK120000625': ['ECK120000625']},"[{'name': 'nagBAC-umpH', 'RegulonDB ID': 'ECK1...",{'ECK125162049': ['ECK120000625']},"[{'name': 'PhoP'}, {'name': 'NagC'}, {'name': ...","{'NagC': ['ECK125162049'], 'PhoP': ['ECK125162...","[{'name': 'Galactose metabolism'}, {'name': 'A...","{'Galactose metabolism': ['ECK125162049'], 'Am...",[{'name': 'Carbohydrate transport and metaboli...,{'Carbohydrate transport and metabolism': ['EC...,"[{'name': 'NagC/TyrR'}, {'name': 'NagC/TyrR'}]",{'NagC/TyrR': ['nagBAC-umpH']},"[{'name': '3.5.1.25'}, {'name': '3.5.1.25'}]","{'3.5.1.25': ['ECK125162049', 'ECK125162049']}",[{'name': 'N-acetylglucosamine-6-phosphate dea...,{'N-acetylglucosamine-6-phosphate deacetylase ...,42 celsius,glucose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
1,10,42C,1,124,1,1,1.0,1308318,SNP,G→C,R110G (CGT→GGT),clsA,NaN,1 124 1 1,True,"(1308318, 1308318)",{ECK120001556},"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",False,False,{},{},{},{},{},True,"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",{'ECK120001556': ['ECK120001556']},"[{'name': 'clsA-yciU', 'RegulonDB ID': 'ECK125...",{'ECK125147415': ['ECK120001556']},[],{},[],{},[{'name': 'Lipid transport and metabolism'}],{'Lipid transport and metabolism': ['ECK125147...,[],{},[],{},[],{},42 celsius,glucose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
2,11,42C,1,124,1,1,1.0,3815859,DEL,Δ82 bp,None,rph,NaN,1 124 1 1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK120000854'], 'ECK1200007...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000079...",[],{},"[{'name': 'Pyrimidine metabolism'}, {'name': '...","{'Pyrimidine metabolism': ['ECK120014627'], 'D...",[{'name': 'Nucleotide transport and metabolism'}],{'Nucleotide transport and metabolism': ['ECK1...,"[{'name': 'purR-KO'}, {'name': 'purR-KO'}]","{'purR-KO': ['rph-pyrE', 'rph-pyrE']}","[{'name': '2.4.2.10'}, {'name': '2.4.2.10'}]","{'2.4.2.10': ['ECK120014627', 'ECK120014627']}",[{'name': 'Orotate phosphoribosyltransferase (...,{'Orotate phosphoribosyltransferase (EC 2.4.2....,42 celsius,glucose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
3,12,42C,1,124,1,1,1.0,4187550,SNP,C→T,A734V (GCG→GTG),rpoC,NaN,1 124 1 1,True,"(4187550, 4187550)",{ECK120000886},"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",False,False,{},{},{},{},{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[],{},"[{'name': 'Purine metabolism'}, {'name': 'Pyri...","{'Purine metabolism': ['ECK120016992'], 'Pyrim...",[{'name': 'Transcription'}],{'Transcription': ['ECK120016992']},[{'name': 'translation'}],{'translation': ['rplKAJL-rpoBC']},"[{'name': '2.7.7.6'}, {'name': '2.7.7.6'}]","{'2.7.7.6': ['ECK120

In [11]:
unique_condition_set = set()
for cond_cat in METADATA_CATEGORIES_FOR_ASSOCIATIONS:
    unique_condition_set |= set(exp_target_cond_df[cond_cat].unique())
unique_condition_set

{'(NH4)2SO4(1)',
 '(NH4)2SO4(1) MgSO4',
 '(NH4)2SO4(2)',
 '(NH4)2SO4(2) MgSO4(0.12)',
 '2 3-butanediol',
 '30 celsius',
 '37 celsius',
 '42 celsius',
 '511145',
 'B. melitensis pgi',
 'C13H10N2O(0.168184)',
 'C6H5FeO7(0.00049) casamino(5) WT P. aeruginosa toxins',
 'C6H5FeO7(0.00049) casamino(5) pvdJ KO P. aeruginosa toxins',
 'CaCl2(0.1)',
 'CaCl2(0.1mM)',
 'D-2-deoxyribose',
 'D-arabinose',
 'D-lyxose',
 'E. coli K-12 MG1655 pCA24N-fucO',
 'FeSO4(20uM) sodium citrate(10mM)',
 'GMOS',
 'H. sapien pgi',
 'H. sapien tpiA',
 'JCL260',
 'K2HPO4(5.3) KH2PO4(2)',
 'KH2PO4(13.6)',
 'KH2PO4(3) Na2HPO4(6.78)',
 'KH2PO4(3) Na2HPO4(6.8)',
 'LB',
 'M63',
 'M9',
 'MS57',
 'MgSO4(0.24)',
 'MgSO4(1mM)',
 'NH4Cl(1)',
 'NaCl(0.5g/L) trace elements',
 'P. aerophilum pgi',
 'P. aerophilum tpiA',
 'PQ(0.8mM)',
 'U-13C glucose(2)',
 'V. cholerae pgi',
 'V. cholerae tpiA',
 'WT',
 'acetate(4)',
 'adipic acid',
 'bla-TEM-1',
 'ceftazidime',
 'chloramphenicol',
 'chloramphenicol(25)',
 'coumaric acid',
 'dox

In [12]:
feat_cond_df = pd.DataFrame()
for _, r in exp_target_cond_df.iterrows():
    for feat in r[FEAT_ANNOT]:
        for cond_col in METADATA_CATEGORIES_FOR_ASSOCIATIONS:
            if r[cond_col] in unique_condition_set:   # I'm not quite sure why I had to check this in the legacy code.
                feat_cond_df = feat_cond_df.append({"feature": feat["name"], "condition":r[cond_col]}, ignore_index=True)
display(len(feat_cond_df), feat_cond_df.head())

93340

,condition,feature
0,42 celsius,nagA
1,glucose(4),nagA
2,NaCl(0.5g/L) trace elements,nagA
3,WT,nagA
4,511145,nagA


TODO: Now that this logic is being used for multiple feature types (therefore multiple times), should extract it into a util script file

In [13]:
from statsmodels.stats import multitest
import scipy


def get_contingency_table(count_mat, row_name, col_name):
    row_col_count = count_mat.loc[row_name, col_name]
    row_not_col_sum_count = count_mat.loc[row_name].sum() - row_col_count
    not_row_col_sum_count = count_mat.T.loc[col_name].sum() - row_col_count
    all_mat_sum_count = count_mat.sum(axis=1).sum()
    not_row_not_col_sum_count = all_mat_sum_count - \
        row_col_count - row_not_col_sum_count - not_row_col_sum_count
    contingency_table = [
        [row_col_count, row_not_col_sum_count],
        [not_row_col_sum_count, not_row_not_col_sum_count]
    ]
    return contingency_table


test_df = pd.DataFrame([[1, 2, 3],
                        [4, 5, 6]],
                       index=["y1", "y2"],
                       columns=["x1", "x2", "x3"])
contingency_table = get_contingency_table(test_df, "y2", "x3")
expected_contingency_table = [[6, 9], [3, 3]]
assert(contingency_table == expected_contingency_table)


def get_multiple_hypothesis_correction(pval_df):
    # Dataframe seemed to be designed to iterate by column more easily than row (see df.iteritems())
    # Have to combine all p-value columns because multitest.multipletests only takes 1D array inputs.
    pval_l = []
    for col in list(pval_df.columns.values):
        pval_l += list(pval_df[col])
    corrected_pval_result = multitest.multipletests(
        pvals=pval_l,
        alpha=ASSOC_ALPHA,
        method=MULTI_HYP_CORR_METHOD)

    # The following splits the multitest.multipletests 1D array output into the same shape, though transposed.
    corrected_pval_l = corrected_pval_result[1]
    df_col_len = pval_df.shape[0]
    corrected_pval_mat = [corrected_pval_l[i:i+df_col_len]
                          for i in range(0, len(corrected_pval_l), df_col_len)]

    # building df with new p-values
    corrected_pval_df = pval_df.copy()
    for outer_l_idx in range(0, len(corrected_pval_df.columns)):
        for inner_l_idx in range(0, len(corrected_pval_df)):
            corrected_pval = corrected_pval_mat[outer_l_idx][inner_l_idx]
            corrected_pval_df.iloc[inner_l_idx, outer_l_idx] = corrected_pval
    return corrected_pval_df


# def get_enrich_genetic_target_df(cond_col, mut_df):
def get_enrich_genetic_target_df(feat_cond_df):
    # Creates tables of counts between each mutation and unique condition.
    cross_counts_df = pd.crosstab(
        feat_cond_df["feature"], feat_cond_df["condition"]
    )

    # To reuse the same DF indeces without having to remake it.
    enrich_odds_ratio_df = cross_counts_df.copy()
    enrich_pvals_df = cross_counts_df.copy()

    for cond in cross_counts_df.columns.values:
        for mut_target in cross_counts_df.index:
            contingency_table = get_contingency_table(cross_counts_df, mut_target, cond)
            odds_ratio, p_val = scipy.stats.fisher_exact(contingency_table, alternative="greater")
            enrich_odds_ratio_df.loc[mut_target, cond] = odds_ratio
            enrich_pvals_df.loc[mut_target, cond] = p_val

    enriched_pvals_df = get_multiple_hypothesis_correction(enrich_pvals_df)

    return enrich_odds_ratio_df, enrich_pvals_df

In [14]:
enrich_odds_ratio_df, enrich_pvals_df = get_enrich_genetic_target_df(feat_cond_df)

In [15]:
signif_genomic_feat_cond_json = []
mut_feat_cond_assoc_json = []
for (mut_serial, row) in enrich_pvals_df.iterrows():
    for cond in row.index:
        p_val = enrich_pvals_df.at[mut_serial, cond]
        odds_ratio = enrich_odds_ratio_df.at[mut_serial, cond]
        mut_feat_cond_assoc_json.append({"mutated features": mut_serial, "condition": cond, "odd ratio": odds_ratio, "p value":p_val})
        if odds_ratio > 1 and p_val < ASSOC_ALPHA:
            signif_genomic_feat_cond_json.append({"mutated features": mut_serial, "condition": cond, "odd ratio": odds_ratio, "p value":p_val})

In [16]:
assoc_df = pd.DataFrame()
for d in mut_feat_cond_assoc_json:
    assoc_df = assoc_df.append(d, ignore_index=True)
assoc_df = assoc_df.set_index("mutated features")
assoc_df.to_csv("./data/supp/mut_geno_cond_assocs.csv")

In [17]:
import pickle
f = open("./data/signif_genomic_feat_cond_json.pkl", 'wb')
pickle.dump(signif_genomic_feat_cond_json, f)
f.close()

In [18]:
signf_assoc_feat_cond_d = {d["mutated features"]:set() for d in signif_genomic_feat_cond_json}
for d in signif_genomic_feat_cond_json:
    target = d["mutated features"]
    condition = d["condition"]
    if target in signf_assoc_feat_cond_d.keys():
        signf_assoc_feat_cond_d[target].add(condition)
    else:
        signf_assoc_feat_cond_d[target] = {condition}
signf_assoc_feat_cond_d

{'C0343': {'LB', 'bla-TEM-1', 'ceftazidime', 'pBGT', 'ΔtpiA'},
 'C0362': {'P. aerophilum tpiA'},
 'C0465': {'30 celsius',
  'E. coli K-12 MG1655 pCA24N-fucO',
  'chloramphenicol(25)',
  'ethylene glycol(30mM)'},
 'aaeR': {'JCL260', 'LB', 'glucose(3)', 'isobutanol(8g/L)'},
 'aas': {'gentamycin(0.03)'},
 'aat': {'hexamethylenediamine'},
 'abgA': {'LB', 'bla-TEM-1', 'ceftazidime', 'pBGT', 'ΔtpiA'},
 'abgB': {'LB', 'bla-TEM-1', 'ceftazidime', 'pBGT', 'ΔtpiA'},
 'abgR': {'LB', 'bla-TEM-1', 'ceftazidime', 'pBGT', 'ΔtpiA'},
 'abgR attenuator terminator': {'LB',
  'bla-TEM-1',
  'ceftazidime',
  'pBGT',
  'ΔtpiA'},
 'abgR terminator': {'LB', 'bla-TEM-1', 'ceftazidime', 'pBGT', 'ΔtpiA'},
 'abgT': {'CaCl2(0.1mM)',
  'KH2PO4(3) Na2HPO4(6.78)',
  'LB',
  'M63',
  'MS57',
  'MgSO4(1mM)',
  'NaCl(0.5g/L) trace elements',
  'bla-TEM-1',
  'ceftazidime',
  'glucose(2)',
  'pBGT',
  'ΔtpiA'},
 'abpA': {'hexanoic acid'},
 'abpA/yfjM': {'glutaric acid'},
 'aceE': {'PQ(0.8mM)', 'glucose-evolved BOP1000'},

In [19]:
def _get_feat_signf_assoc_cond_set(feat_d, mut_conds):
    feat_signf_assoc_cond_set = set()
    if feat_d["name"] in signf_assoc_feat_cond_d.keys():
        
        # Only includes conditions from exp mut comes from, therefore may be subset of signf assoc cond.
        for c in signf_assoc_feat_cond_d[feat_d["name"]]:
            if c in mut_conds:
                feat_signf_assoc_cond_set.add(c)
    
    return feat_signf_assoc_cond_set


def get_feat_signf_assoc_cond_set_json(mut_df_row, feat_annot_name):
    feat_json = []
    for d in mut_df_row[feat_annot_name]:
        feat_d = d.copy()
        mut_conds = list(mut_df_row[METADATA_CATEGORIES_FOR_ASSOCIATIONS])
        feat_d["significantly associated conditions"] = _get_feat_signf_assoc_cond_set(d, mut_conds)
        feat_json.append(feat_d)
    return feat_json


exp_target_cond_df[FEAT_ANNOT] = exp_target_cond_df.apply(lambda r: get_feat_signf_assoc_cond_set_json(r, FEAT_ANNOT), axis=1)

In [20]:
exp_target_cond_df.to_pickle("./data/4_df.pkl")
exp_target_cond_df.shape

(5676, 52)